# Read prediction results from .pkl file

In [95]:
import pickle
import random
import numpy as np
import matplotlib.pyplot as plt

path = '../rollouts/Fragment/Step-0-100-3/rollout_12.pkl'

with open(path, "rb") as file:
    rollout_data = pickle.load(file)

init_pos = rollout_data['initial_positions']
pred_pos = rollout_data['predicted_rollout']
gt_pos = rollout_data['ground_truth_rollout']
gt_pos = np.concatenate((init_pos, gt_pos), axis=0)
id = int(path.split('.p')[0].split('_')[-1])
pred_pos = np.concatenate((init_pos, pred_pos), axis=0)
case = rollout_data["metadata"]["file_test"][id].split('.n')[0]
print(f"{case}, shape: {gt_pos.shape}")

009_120_6_0.4C30, shape: (34, 246248, 3)


# Compute last timestep fragments based on DIST_THRES and Z_THRES

In [96]:
import numpy as np
import networkx as nx
from scipy.spatial.distance import cdist
from scipy.spatial import ConvexHull

DIST_THRES = 10.1
Z_THRES = 140

def compute_fragment_by_graph(trajectory):
    
    # assuming `trajectory` is a numpy array of shape (mtimestep, nparticles, 3)
    last_positions = trajectory[-1, :, :]
    prev_positions = trajectory[-2, :, :]
    
    # calculate speed of each particle
    velocities = np.linalg.norm(last_positions - prev_positions, axis=1)

    # filter out particles with speed below the threshold
    mask = last_positions[:, 2] > Z_THRES
    filtered_positions = last_positions[mask]
    filtered_vels = velocities[mask]

    # adjacency matrix, initially all particles are connected
    adjacency_matrix = cdist(filtered_positions, filtered_positions) < DIST_THRES

    # create graph from adjacency matrix
    G = nx.from_numpy_array(adjacency_matrix)

    # find fragments as connected components
    fragments = list(nx.connected_components(G))
    
    return fragments, filtered_positions, filtered_vels

gt_fragments, gt_filtered_positions, gt_filtered_vels = compute_fragment_by_graph(gt_pos)

pred_fragments, pred_filtered_positions, pred_filtered_vels = compute_fragment_by_graph(pred_pos)

print(f"Distance threshold: {DIST_THRES}, Z threshold: {Z_THRES}")
print(f"Total number of fragments: gt-{len(gt_fragments)}, pred-{len(pred_fragments)}")

Distance threshold: 10.1, Z threshold: 140
Total number of fragments: gt-2966, pred-2914


# Visualise fragment in the last timestep

In [ ]:
import plotly.graph_objects as go
import plotly.io as pio
import pandas as pd
import trimesh
from scipy.spatial import ConvexHull
import numpy as np
from matplotlib import cm
from pathlib import Path

out_dir = f"../rollouts/Fragment/fragmentation/{case}"
Path(out_dir).mkdir(parents=True, exist_ok=True)

positions = pred_filtered_positions
velocities = pred_filtered_vels
fragments = pred_fragments

xmin, ymin, zmin = positions.min(axis=0)
xmax, ymax, zmax = positions.max(axis=0)

v_min = 0
v_max = 100

fig = go.Figure(data=[go.Scatter3d(
    x=positions[:, 0],
    y=positions[:, 1],
    z=positions[:, 2],
    mode='markers',
    marker=dict(
        size=1,
        color='grey',
        opacity=0.8
    )
)])

# Define the conditions for fragment filtering
min_particles = 3

# Counter for number of fragments
num_fragments = 0

# Loop over fragments
for i, fragment in enumerate(fragments):
    fragment_positions = positions[list(fragment)]
    fragment_velocities = velocities[list(fragment)]

    # Check if this fragment satisfies the conditions
    if len(fragment) > min_particles:
        num_fragments += 1

        # Compute the average normalized velocity for this fragment
        avg_velocity = np.mean(fragment_velocities)

        # Map the average velocity to a color
        color_rgb = cm.jet(avg_velocity / v_max)[:3] # changed to jet colormap
        color_rgba = f"rgba({color_rgb[0]*255}, {color_rgb[1]*255}, {color_rgb[2]*255}, 0.8)"

        # Your existing code to create a Trimesh object
        mesh = trimesh.Trimesh(vertices=fragment_positions, process=True)

        # Use the vertices and faces of the entire mesh instead of the convex hull
        vertices = mesh.vertices
        faces = mesh.faces

        # Add the mesh to the figure
        fig.add_trace(go.Mesh3d(
            x=vertices[:, 0],
            y=vertices[:, 1],
            z=vertices[:, 2],
            i=faces[:, 0],
            j=faces[:, 1],
            k=faces[:, 2],
            color=color_rgba,
            intensity=[avg_velocity]*faces.shape[0],
            colorscale='Jet',
            cmin=v_min,
            cmax=v_max,
            showscale=True,
        ))

fig.update_scenes(
    xaxis=dict(range=[xmin, xmax]), 
    yaxis=dict(range=[ymin, ymax]), 
    zaxis=dict(range=[zmin, zmax])
)

# Update layout with the title
fig.update_layout(
    autosize=False,
    width=900,
    height=900,
    scene=dict(
        aspectmode='manual',
        aspectratio=dict(x=1, y=1, z=0.5),
        camera = dict(
            up=dict(x=0, y=0, z=1),  # this is the 'up' direction for the camera
            center=dict(x=0, y=0, z=0),  # this will move the camera itself
            eye=dict(x=1.2, y=1.2, z=1.2)  # this moves the 'eye' of the camera
        )
    ),
    margin=dict(l=0, r=0, b=0, t=0),  # tight layout
    title=dict(
        text=f"fragments: {num_fragments}",
        x=0.5,
        y=0.9,
        xanchor='center',
        yanchor='top',
    ),
)
fig.show()

# Compute fragment for multiple steps, generating fragments as a list

In [55]:
import numpy as np
import networkx as nx
from scipy.spatial.distance import cdist
from scipy.spatial import ConvexHull

DIST_THRES = 10.1
Z_THRES = 145

def compute_fragment_by_graph(trajectory):
    
    fragments_list, filtered_positions_list, filtered_vels_list = [], [], []
    
    for step in range(10, 34):
        # assuming `trajectory` is a numpy array of shape (mtimestep, nparticles, 3)
        last_positions = trajectory[step, :, :]
        prev_positions = trajectory[step-1, :, :]

        # calculate speed of each particle
        velocities = np.linalg.norm(last_positions - prev_positions, axis=1)

        # filter out particles with speed below the threshold
        mask = last_positions[:, 2] > Z_THRES
        filtered_positions = last_positions[mask]
        filtered_vels = velocities[mask]

        # adjacency matrix, initially all particles are connected
        adjacency_matrix = cdist(filtered_positions, filtered_positions) < DIST_THRES

        # create graph from adjacency matrix
        G = nx.from_numpy_array(adjacency_matrix)

        # find fragments as connected components
        fragments = list(nx.connected_components(G))
        
        fragments_list.append(fragments)
        filtered_positions_list.append(filtered_positions)
        filtered_vels_list.append(filtered_vels)
        
    return fragments_list, filtered_positions_list, filtered_vels_list

gt_fragments, gt_filtered_positions, gt_filtered_vels = compute_fragment_by_graph(gt_pos)
pred_fragments, pred_filtered_positions, pred_filtered_vels = compute_fragment_by_graph(pred_pos)

print(f"Distance threshold: {DIST_THRES}, Z threshold: {Z_THRES}")
print(f"Total number of fragments: gt-{len(gt_fragments)}, pred-{len(pred_fragments)}")

Distance threshold: 10.1, Z threshold: 145
Total number of fragments: gt-24, pred-24


# Generate images for the fragmentation process

In [69]:
import plotly.graph_objects as go
import plotly.io as pio
import pandas as pd
import trimesh
from scipy.spatial import ConvexHull
import numpy as np
from matplotlib import cm
from pathlib import Path

# Define the conditions for fragment filtering
min_particles = 3
mode = 'pred'

positions = gt_filtered_positions[-1]
xmin, ymin, zmin = positions.min(axis=0)
xmax, ymax, zmax = positions.max(axis=0)
v_min = 0.
v_max = 6.
    
out_dir = f"../rollouts/Fragment/fragmentation/{case}"
Path(out_dir).mkdir(parents=True, exist_ok=True)

for step in range(24):
    if mode == 'gt':
        positions = gt_filtered_positions[step]
        velocities = gt_filtered_vels[step]
        fragments = gt_fragments[step]
    else:
        positions = pred_filtered_positions[step]
        velocities = pred_filtered_vels[step]
        fragments = pred_fragments[step]

    fig = go.Figure(data=[go.Scatter3d(
        x=positions[:, 0],
        y=positions[:, 1],
        z=positions[:, 2],
        mode='markers',
        marker=dict(
            size=1,
            color=velocities,  # Use the normalized velocities here
            colorscale='Jet',
            cmin=v_min,
            cmax=v_max,
            opacity=0.8
        )
    )])

    # Counter for number of fragments
    num_fragment_particles = 0

    # Loop over fragments
    for i, fragment in enumerate(fragments):
        fragment_positions = positions[list(fragment)]
        fragment_velocities = velocities[list(fragment)]
        num_fragment_particles += len(fragment)
        
        # Check if this fragment satisfies the conditions
        if len(fragment) > min_particles:
            # Compute the average normalized velocity for this fragment
            avg_velocity = np.mean(fragment_velocities)

            # Map the average velocity to a color
            color_rgb = cm.jet(avg_velocity / v_max)[:3] # changed to jet colormap
            color_rgba = f"rgba({color_rgb[0]*255}, {color_rgb[1]*255}, {color_rgb[2]*255}, 0.8)"

            mesh = trimesh.Trimesh(vertices=fragment_positions, process=True)
            hull = mesh.convex_hull

            # Then add the mesh to the figure
            fig.add_trace(go.Mesh3d(
                x=hull.vertices[:, 0],
                y=hull.vertices[:, 1],
                z=hull.vertices[:, 2],
                i=hull.faces[:, 0],
                j=hull.faces[:, 1],
                k=hull.faces[:, 2],
                color=color_rgba,
                intensity=[avg_velocity]*hull.faces.shape[0],
                colorscale='Jet',
                cmin=v_min,
                cmax=v_max,
                showscale=True,
            ))
    fragment_mass = num_fragment_particles * 0.0024
            
    fig.update_scenes(
        xaxis=dict(range=[xmin, xmax]), 
        yaxis=dict(range=[ymin, ymax]), 
        zaxis=dict(range=[zmin, zmax])
    )

    # Update layout with the title
    fig.update_layout(
        autosize=False,
        width=2560,
        height=1440,
        scene=dict(
            xaxis=dict(title='X', title_font=dict(size=22), tickfont=dict(size=16)),
            yaxis=dict(title='Y', title_font=dict(size=22), tickfont=dict(size=16)),
            zaxis=dict(title='Z', title_font=dict(size=22), tickfont=dict(size=16)),
            aspectmode='manual',
            aspectratio=dict(x=1, y=1, z=0.5),
            camera = dict(
                up=dict(x=0, y=0, z=1),  # this is the 'up' direction for the camera
                center=dict(x=0, y=0, z=0),  # this will move the camera itself
                eye=dict(x=1.5, y=1.5, z=0.3)  # this moves the 'eye' of the camera
            )
        ),
        margin=dict(l=0, r=0, b=0, t=0),  # tight layout
        title=dict(
            text=f"Step: {step:02}, time: {(step+10)*0.06:.3f} ms, fragment mass: {fragment_mass:.3f} kg",
            x=0.5,
            y=0.9,
            xanchor='center',
            yanchor='top',
        ),
    )
    
    file_name = f"{mode}-{step:02}.png"
    save_path = Path(out_dir) / file_name
    fig.write_image(str(save_path), scale=1)

In [81]:
velocities.max()*100/6

89.9348258972168